In [1]:
# setup
from pyswmm import Simulation
import os, dask, time, shutil, numpy as np
from pyswmm.lib import DLL_SELECTION

In [2]:
# save absolute path of input file location
# this string is how we will tell pyswmm where to locate the files it needs to run its simulations
input_path = os.path.abspath("../probabilistic_python/input/swmm/input_")
print(input_path)

# the name of the dll path we base the names of the other dll paths on. 
dll_path = DLL_SELECTION()
print(dll_path)

# create 3 new vehicles for swmm tasks by copying the original one, located at dll_path
for i in range(1,4):
    # create path to the .exe we are going to create
    new_dll_path = dll_path[:-4]+'-'+str(i)+dll_path[-4:]
    shutil.copyfile(dll_path, new_dll_path)

# A list of flags indicating if their respective vehicle if available
dlls_available=[True,True,True]

C:\Users\Julia Stelman\Desktop\Watershed\chelsvig_urban_pesticides\probabilistic_python\input\swmm\input_
C:/ProgramData/Anaconda3/envs/epa/lib/site-packages/pyswmm/lib/windows/swmm5-x64.dll


In [4]:
'''
Makes arrangements for executing the following task:
    Run specified probabilistic simulations in SWMM from the specified .inp file. Save output in equivalently named .out file.
Inputs:
    i <integer> -indicates input folder id number (for now, will only ever be 1-5).-
Immediate Output:
    <dask.delayed.Delayed> -arrangements for executing the task when commanded at a future time (by the 'compute()' function)
Computable Output:
    <string> -message indicating that the simulation has been completed on the .inp file in the 'i' input folder.-
'''
# this decorator is the first step in using dask to parallelize swmm simulations
@dask.delayed
def delay_job(i):
    # specify the directory with the file pyswmm needs by attaching the folder id to the rest of the folder's absolute path
    sim_folder = input_path + str(i)
    # specify the actual file pyswmm needs
    sim_file = os.path.join(sim_folder, r'JS_NPlesantCreek.inp')
    print("Simulation input file found:", sim_file)
    # specify the file that pyswmm will (over)write with output after running the probabilistic simulation
    binary_file = sim_folder + r'\JS_NPlesantCreek.out'
    # delete pre-existing .out, if present, in order to run swmm agreeably
    if os.path.exists(binary_file):
        print("Deleting current copy of <JS_NPlesantCreek.out> so new copy can be created.")
        os.remove(binary_file)
    # stagger starting times 2 sec apart
    time.sleep(2*i)
    # use the first available exe file to run
    while not any(dlls_available):
        time.sleep(1.3)
    # find the one that's available
    dllno = np.where([dlls_available])[0][0]
    # mark it unavailable
    dlls_available[dllno]=False
    # dll attempting to use. If not occupied, we will succeed. OW...
    lib_path = dll_path[:-4]+'-'+str(dllno+1)+dll_path[-4:]
    # load the model {no interaction, write (binary) results to <JS_NPlesantCreek.out>, use the specified dll}
    sim = Simulation(inputfile=sim_file, reportfile=None, outputfile=binary_file, swmm_lib_path=lib_path)
    # simulate the loaded model
    sim.execute()
    # delete the .exe file
    #os.remove(lib_path)
    # a message to indicate success
    return("file " + str(i) + " simulated!")

In [5]:
# can it do multiple?
delayed_jobs = [delay_job(1),
                delay_job(2),
                delay_job(3),
                delay_job(4),
                delay_job(5)]
delayed_jobs

[Delayed('delay_job-bd3baf06-b872-403b-971b-46a3672c88cc'),
 Delayed('delay_job-934ab6e0-42b8-4b9f-93cf-5419a85b502c'),
 Delayed('delay_job-a4989038-bfc1-44c5-9d71-12eb76921501'),
 Delayed('delay_job-802b927b-bd89-4ee4-bcef-3a537ff15370'),
 Delayed('delay_job-668f2bd5-c590-486d-ace9-9b28bdb92a81')]

In [6]:
%%time
dask.delayed(print)(delayed_jobs).compute()

Simulation input file found: C:\Users\Julia Stelman\Desktop\Watershed\chelsvig_urban_pesticides\probabilistic_python\input\swmm\input_5\JS_NPlesantCreek.inp
Simulation input file found: C:\Users\Julia Stelman\Desktop\Watershed\chelsvig_urban_pesticides\probabilistic_python\input\swmm\input_4\JS_NPlesantCreek.inp
Simulation input file found: C:\Users\Julia Stelman\Desktop\Watershed\chelsvig_urban_pesticides\probabilistic_python\input\swmm\input_2\JS_NPlesantCreek.inp
Simulation input file found: C:\Users\Julia Stelman\Desktop\Watershed\chelsvig_urban_pesticides\probabilistic_python\input\swmm\input_3\JS_NPlesantCreek.inp
Simulation input file found: C:\Users\Julia Stelman\Desktop\Watershed\chelsvig_urban_pesticides\probabilistic_python\input\swmm\input_1\JS_NPlesantCreek.inp
Deleting current copy of <JS_NPlesantCreek.out> so new copy can be created.

... SWMM Version 5.3.0.dev0
... Run Complete

SWMMException: 
  ERROR 401: general system error.


... SWMM Version 5.3.0.dev0

PySWMM can handle simultaneously running two parallel processes, as long as each one uses its own direct link library. 